O seguinte Código realiza um Webscraping de filmes em um site de avaliações, gerando um dataset com os filmes mais bem avaliados de diversos gêneros. Por fim, fazendo um sorteio de filmes neste dataset filtrado por gênero.

Bibliotecas:

In [1]:
import pandas as pd
import random
from bs4 import BeautifulSoup
import requests
from csv import writer

URL dos principais gêneros:

In [2]:
url_ação = 'https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=E76QSXP2Y9EEHKBVZRNZ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1'
url_aven = 'https://www.imdb.com/search/title/?genres=adventure&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=E76QSXP2Y9EEHKBVZRNZ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_2'
url_anim = 'https://www.imdb.com/search/title/?genres=animation&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=E76QSXP2Y9EEHKBVZRNZ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_3'
url_come = 'https://www.imdb.com/search/title/?genres=comedy&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=E76QSXP2Y9EEHKBVZRNZ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_5'
url_dram = 'https://www.imdb.com/search/title/?genres=drama&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=E76QSXP2Y9EEHKBVZRNZ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_7'
url_fant = 'https://www.imdb.com/search/title/?genres=fantasy&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=E76QSXP2Y9EEHKBVZRNZ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_9'
url_horr = 'https://www.imdb.com/search/title/?genres=horror&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=E76QSXP2Y9EEHKBVZRNZ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_12'
url_roma = 'https://www.imdb.com/search/title/?genres=romance&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=E76QSXP2Y9EEHKBVZRNZ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_16'
url_scif = 'https://www.imdb.com/search/title/?genres=sci_fi&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=E76QSXP2Y9EEHKBVZRNZ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_17'
url_thri = 'https://www.imdb.com/search/title/?genres=thriller&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=E76QSXP2Y9EEHKBVZRNZ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_19'

urls = [url_ação, url_aven, url_anim, url_come, url_dram, url_fant, url_horr, url_roma, url_scif, url_thri]

Gerando uma tabela com os filmes:

In [3]:
with open('filmes.csv', 'w', encoding='utf8', newline='') as f:
    thewriter = writer(f)
    header = ['Título', 'Ano', 'Duração', 'Gênero', 'Nota IMDb']
    thewriter.writerow(header)

    for j in urls:   
        page = requests.get(j)
        soup = BeautifulSoup(page.content, 'html.parser')
        list = soup.find_all('div', class_='lister-item-content')

        for i in list:    
            título = i.find('a').text.replace('\n', '')
            ano = i.find('span', class_='lister-item-year text-muted unbold').text.replace('\n', '')
            duração = i.find('span', class_='runtime').text.replace('\n', '')
            gênero = i.find('span', class_='genre').text.replace('\n', '').strip()
            nota = i.find('div', class_='inline-block ratings-imdb-rating').text.replace('\n', '')

            info = [título, ano, duração, gênero, nota]
            thewriter.writerow(info)

Preparando o dataset:

In [4]:
df = pd.read_csv('filmes.csv')
df.drop_duplicates(inplace=True)
df.head()

,Título,Ano,Duração,Gênero,Nota IMDb
0,Homem-Aranha: Através do Aranhaverso,(2023),140 min,"Animation, Action, Adventure",9.1
1,Batman: O Cavaleiro das Trevas,(2008),152 min,"Action, Crime, Drama",9.0
2,O Senhor dos Anéis: O Retorno do Rei,(2003),201 min,"Action, Adventure, Drama",9.0
3,A Origem,(2010),148 min,"Action, Adventure, Sci-Fi",8.8
4,O Senhor dos Anéis: A Sociedade do Anel,(2001),178 min,"Action, Adventure, Drama",8.8


Recomendando filmes com do gênero desejado:

In [5]:
Gêneros = ['Action', 'Adventure', 'Animation', 'Comedy', 'Drama', 'Fantasy', 'Horror', 'Romance', 'Sci-Fi', 'Thriller']

g = input(print('Gênero desejado (em inglês):'))

df2 = df[df['Gênero'].str.contains(g)]
df2.sample(4)

Gênero desejado (em inglês):


,Título,Ano,Duração,Gênero,Nota IMDb
44,V de Vingança,(2005),132 min,"Action, Drama, Sci-Fi",8.2
404,Star Wars: Episode V - The Empire Strikes Back,(1980),124 min,"Action, Adventure, Fantasy",8.7
3,A Origem,(2010),148 min,"Action, Adventure, Sci-Fi",8.8
426,Blade Runner,(1982),117 min,"Action, Drama, Sci-Fi",8.1
